<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Datenaufbereitung" data-toc-modified-id="Datenaufbereitung-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Datenaufbereitung</a></span></li><li><span><a href="#Datenauswertung" data-toc-modified-id="Datenauswertung-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Datenauswertung</a></span></li></ul></div>

In diesem Notebook werden die zuvor aufbereiteten Abstimmungsdaten ausgewertet. 

## Datenaufbereitung

In [1]:
# Die nötigen Bibliotheken werden importiert. 
import pandas as pd
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

In [2]:
# Die Daten werden eingelesen. 
df = pd.read_csv('daten/abstimmungen_grossrat.csv', index_col='Unnamed: 0', low_memory=False)

In [20]:
# Für jede Vorlage wird die "Parteilinie" bestimmt. 
df_parteilinie = df.groupby(by=['geschäftsname', 'dateiname', 'register', 'partei'])['stimme'].value_counts().to_frame()
df_parteilinie = df_parteilinie.unstack()
df_parteilinie['stimme'] = df_parteilinie['stimme'].fillna(0)

def linienfinder(ja, nein, enthaltung, abwesend): 
    t = ja + nein
    if ja / t >= 0.666666:
        return 'ja'
    elif nein / t >= 0.666666:
        return 'nein'
    else:
        return 'unentschieden'
    
for i, r in tqdm(df_parteilinie.iterrows(),total=len(df_parteilinie), leave=False):
    l = linienfinder(r['stimme']['ja'], r['stimme']['nein'], r['stimme']['enthaltung'], r['stimme']['abwesend'])
    df_parteilinie.at[i, 'linie'] = l
df_parteilinie = df_parteilinie[['linie']].reset_index()

df_temp = df_parteilinie.groupby(by=['geschäftsname', 'dateiname', 'register', 'partei']).sum()[['linie']]
df_temp = df_temp.unstack()
df_temp.columns = [x[2] for x in df_temp.columns]
df_parteilinie = df_temp.copy()
df_parteilinie.head()

AL Die Mitte   EDU  \
geschäftsname dateiname                  register                          
#0000.0000_0  Frühlingssession 2019.xlsx 4.03.2019   NaN        ja    ja   
              Frühlingssession 2020.xlsx 2.03.2020   NaN        ja    ja   
                                         4.03.2020   NaN      nein    ja   
              Herbstsession 2019.xlsx    2.09.2019   NaN        ja    ja   
              Wintersession 2018.xlsx    19.11.2018  NaN      nein  nein   

                                                      EVP   FDP Grüne    SP  \
geschäftsname dateiname                  register                             
#0000.0000_0  Frühlingssession 2019.xlsx 4.03.2019   nein    ja  nein  nein   
              Frühlingssession 2020.xlsx 2.03.2020     ja    ja    ja    ja   
                                         4.03.2020     ja    ja    ja    ja   
              Herbstsession 2019.xlsx    2.09.2019     ja    ja    ja    ja   
              Wintersession 2018.xlsx    19.11.2018    ja  nein    ja    ja   

                                                               SVP  \
geschäftsname dateiname                  register                    
#0000.0000_0  Frühlingssession 2019.xlsx 4.03.2019              ja   
              Frühlingssession 2020.xlsx 2.03.2020            nein   
                                         4.03.2020   unentschieden   
              Herbstsession 2019.xlsx    2.09.2019              ja   
              Wintersession 2018.xlsx    19.11.2018           nein   

                                                    fraktionslos   glp  
geschäftsname dateiname                  register                       
#0000.0000_0  Frühlingssession 2019.xlsx 4.03.2019           NaN    ja  
              Frühlingssession 2020.xlsx 2.03.2020           NaN    ja  
                                         4.03.2020           NaN    ja  
              Herbstsession 2019.xlsx    2.09.2019           NaN    ja  
              Wintersession 2018.xlsx    19.11.2018          NaN  nein

In [4]:
# Ein Datensatz zur "Linientreue" der einzelnen Parlamentarier wird erstellt. 
df_politiker = df.drop_duplicates(subset=['nachname', 'vorname', 'partei'])[['nachname', 'vorname', 'partei']]
df_politiker = df_politiker.dropna(subset=['nachname', 'vorname'])
df_politiker.reset_index(drop=True)

# Parlamentarier, die keiner Partei angehören, werden ignoriert. 
df_politiker = df_politiker[df_politiker['partei'] != 'fraktionslos']
df_politiker.head()

,nachname,vorname,partei
1,Aebi,Markus,SVP
2,Aebischer,Verena,SVP
3,Amstutz,Madeleine,SVP
5,Augstburger,Ueli,SVP
6,Benoit,Roland,SVP


In [26]:
for i, r in tqdm(df_politiker.iterrows(), total=len(df_politiker), leave=False):
    linientreu = 0
    abgewichen = 0

    df_temp = df[(df['vorname'] == r['vorname']) & (df['nachname'] == r['nachname']) & (df['partei'] == r['partei'])]

    for j, s in df_temp.iterrows():
        eingelegt = s['stimme']
        parteilinie = df_parteilinie.loc[s['geschäftsname'], s['dateiname'], s['register']][s['partei']]
        
        if eingelegt == parteilinie:
            linientreu += 1
        else: 
            abgewichen += 1
    df_politiker.at[i, 'linientreu'] = linientreu
    df_politiker.at[i, 'abgewichen'] = abgewichen

df_politiker['linientreue'] = 100 * df_politiker['linientreu'] / df_politiker[['linientreu', 'abgewichen']].sum(axis=1)
df_politiker.head()

,nachname,vorname,partei,linientreu,abgewichen,linientreue
1,Aebi,Markus,SVP,1543.0,356.0,81.253291
2,Aebischer,Verena,SVP,1777.0,144.0,92.503904
3,Amstutz,Madeleine,SVP,656.0,64.0,91.111111
5,Augstburger,Ueli,SVP,1615.0,298.0,84.422373
6,Benoit,Roland,SVP,1666.0,254.0,86.770833


## Datenauswertung

In [30]:
# Welche Politiker politisierten am meisten auf Parteilinie, bzw. wichen nie oder selten von der 2/3-Mehrheit ab?
df_politiker.sort_values(by='linientreue', ascending=False).head(20)

,nachname,vorname,partei,linientreu,abgewichen,linientreue
87,Wüthrich,Andreas,SP,2.0,0.0,100.000000
48,Steiner,Hanspeter,EVP,677.0,33.0,95.352113
48,Rüegsegger,Hans Jörg,SVP,677.0,33.0,95.352113
116,Gerber,Thomas,Grüne,1797.0,93.0,95.079365
164,Baumann-Berger,Katharina,EDU,1823.0,104.0,94.603010
29,Ammann,Christa,AL,419.0,24.0,94.582393
46,Bossard-Jenni,Tabea,EVP,677.0,42.0,94.158554
46,Knutti,Thomas,SVP,677.0,42.0,94.158554
4470,Roulet Romy,Sandra,SP,249.0,16.0,93.962264
10,von Bergen,Margret,EVP,680.0,44.0,93.922652


In [31]:
# Welche Politiker wichen am häufigsten von der Parteilinie ab? 
df_politiker.sort_values(by='linientreue', ascending=True).head(20)

,nachname,vorname,partei,linientreu,abgewichen,linientreue
113144,Kusano,Lea,SP,0.0,243.0,0.000000
32282,Hamdaoui,Mohamed,Die Mitte,688.0,720.0,48.863636
154,Aeschlimann,Martin,EVP,661.0,550.0,54.582989
18,Hofer,Stefan,SVP,836.0,664.0,55.733333
90,von Greyerz,Nicola,SP,1018.0,791.0,56.274185
17,Hess,Erich,SVP,922.0,408.0,69.323308
67,Hässig Vinzens,Kornelia,SP,1340.0,535.0,71.466667
108,Costa,Stefan,FDP,1006.0,390.0,72.063037
30,Ruchti,Fritz,SVP,1348.0,522.0,72.085561
158,Jost,Marc,EVP,875.0,336.0,72.254335


In [36]:
# In welcher Partei gibt es am meisten, in welchen am wenigsten Abweichlerinnen und Abweichler?
df_politiker.groupby(by='partei')[['linientreue']].mean().sort_values(by='linientreue')

,linientreue
partei,
SP,83.213184
Grüne,84.601184
glp,84.962929
Die Mitte,85.113724
EVP,85.577893
SVP,86.164270
FDP,88.322159
EDU,93.125454
AL,94.582393


In [46]:
# Bei welchen Geschäften wichen am meisten Personen von der Parteilinie ab? Bei welchen wich niemand ab?
df_geschäfte = df_parteilinie.index.to_frame()
parteien = df['partei'].unique()

for i,r in tqdmq(df_geschäfte.iterrows(), total):
    for p in parteien: 
        parteilinie = df_parteilinie.loc[i][p]
        
        df_geschäfte.at[i, p + '_parteilinie'] = parteilinie
        
df_geschäfte

ValueError: could not convert string to float: 'unentschieden'

In [13]:
# Ergibt sich evtl. ein regionales Muster hinter Abweichungen? Z.B. Oberländer Grossrätinnen und Grossräte, Emmental usw? (evtl. kann ich diese Frage dann bei der Analyse deiner Auswertungen beantworten).


SyntaxError: invalid syntax (<ipython-input-13-693a438cd2dc>, line 1)

In [ ]:
# Falls das machbar ist: Stimmen Personen, die bei Einzelabstimmungen gegen die Parteilinie abgestimmt haben, dann auch bei der Schlussabstimmung gegen die Parteilinie?